In [1]:
import os
import glob
import numpy as np
import pyvips
from tqdm import tqdm
import re
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

In [4]:
TRUTH_DIR = '/BrainSeg/norm_png/'
BASE_DIR = '/BrainSeg/'
TEST_DIR = '/BrainSeg/Classify_Results/ResNet/GMBData/'
SAVE_IMG_DIR = BASE_DIR + 'Classify_Results/ResNet/GMBMasks_2/'
if not os.path.exists(SAVE_IMG_DIR):
    os.makedirs(SAVE_IMG_DIR)
print(SAVE_IMG_DIR)

/BrainSeg/Classify_Results/ResNet/GMBMasks_2/


In [5]:
test_img = sorted(os.listdir(TRUTH_DIR))
image_names = [imagename.split('.')[0] for imagename in test_img]
image_names

['NA3777-02_AB',
 'NA4077-02_AB',
 'NA4092-02_AB',
 'NA4107-02_AB',
 'NA4160-02_AB',
 'NA4195-02_AB',
 'NA4256-02_AB',
 'NA4299-02_AB',
 'NA4391-02_AB',
 'NA4450-02_AB',
 'NA4463-02_AB',
 'NA4471-02_AB',
 'NA4553-02_AB',
 'NA4626-02_AB',
 'NA4672-02_AB',
 'NA4675-02_AB',
 'NA4691-02_AB',
 'NA4695-02_AB',
 'NA4894-02_AB17-24',
 'NA4907-02_AB17-24',
 'NA4944-02_AB17-24',
 'NA4945-02_AB17-24',
 'NA4967-02_AB17-24',
 'NA4971-02_AB17-24',
 'NA4972-02_AB17-24',
 'NA4992-02_AB17-24',
 'NA4993-02_AB17-24',
 'NA5010-02_AB17-24',
 'NA5015-02_AB17-24',
 'NA5029-02_AB17-24']

In [6]:
for img in image_names:
    nums = np.load(TEST_DIR + img + '.npy')
    nums = np.repeat(nums[:, :, np.newaxis], 3, axis=2)
    #print(nums.shape, nums.dtype)

    # nums[:,:,0] = RED, nums[:,:,1] = Green, nums[:,:,2] = Blue
    idx_1 = np.where(nums[:,:,0] == 2)  # Index of label 1 (WM)
    idx_2 = np.where(nums[:,:,0] == 1)  # Index of label 2 (GM)

    # For label 0, leave as black color
    # For label 1, set to cyan color: R0G255B255
    nums[:,:,0].flat[np.ravel_multi_index(idx_1, nums[:,:,0].shape)] = 0
    nums[:,:,1].flat[np.ravel_multi_index(idx_1, nums[:,:,1].shape)] = 255
    nums[:,:,2].flat[np.ravel_multi_index(idx_1, nums[:,:,2].shape)] = 255
    # For label 2, set to yellow color: R255G255B0
    nums[:,:,0].flat[np.ravel_multi_index(idx_2, nums[:,:,0].shape)] = 255
    nums[:,:,1].flat[np.ravel_multi_index(idx_2, nums[:,:,1].shape)] = 255
    nums[:,:,2].flat[np.ravel_multi_index(idx_2, nums[:,:,2].shape)] = 0

    save_img = Image.fromarray(nums, 'RGB')
    save_img.save(SAVE_IMG_DIR + img + '.png')